In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../Dataset'))
sys.path.append(os.path.abspath('..'))

import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import utils, func_preprocesamiento
importlib.reload(utils)
importlib.reload(func_preprocesamiento)
from utils import stratified_split
from func_preprocesamiento import preprocesar, acotar_caracteristicas


In [79]:
alquileres = pd.read_csv('../Dataset/raw/alquiler_AMBA_dev.csv')
print(alquileres['ITE_ADD_CITY_NAME'].unique())
for ciudad in alquileres['ITE_ADD_CITY_NAME'].unique():
    ciudad_especifico = alquileres[(alquileres['ITE_ADD_CITY_NAME'] == ciudad)]
    if len(ciudad_especifico) >= 1500:
        output_dir = f'../Dataset/Barrios/{ciudad}'
        os.makedirs(output_dir)
        ciudad_especifico.to_csv(f'{output_dir}/{ciudad}_dev.csv')

/tmp/ipykernel_9772/936711182.py:1: DtypeWarning: Columns (8,33,36) have mixed types. Specify dtype option on import or set low_memory=False.
  alquileres = pd.read_csv('../Dataset/raw/alquiler_AMBA_dev.csv')


['Caseros' 'Capital Federal' 'Vicente López' 'Pilar' 'Moreno'
 'Tres de febrero' 'San Isidro' 'San Miguel' 'Lomas de Zamora'
 'La Matanza' 'Quilmes' 'Morón' 'General San Martín' 'San Fernando'
 'Lanús' 'Tigre' 'Esteban Echeverría' 'Avellaneda' 'Castelar'
 'Malvinas Argentinas' 'José C. Paz' 'Berazategui' 'Ezeiza' 'Escobar'
 'Ituzaingó' 'Hurlingham' 'Almirante Brown' 'Florencio Varela' 'Merlo'
 'La Plata' 'Barracas' 'Presidente Perón' 'San Cristóbal' 'Versailles'
 'Villa General Mitre' 'Beccar' 'Saavedra' 'Velez Sarsfield' 'Las Cañitas']


In [80]:
capi = alquileres[(alquileres['ITE_ADD_CITY_NAME']=='Capital Federal')]
comunas = {
    'COMUNA 1': ["Retiro", "San Nicolás", "Puerto Madero", "San Telmo", "Montserrat" , "Constitución"],
    'COMUNA 2':["Recoleta"],
    'COMUNA 3':["Balvanera" , "San Cristóbal"],
    'COMUNA 4':["La Boca", "Barracas", "Parque Patricios" , "Nueva Pompeya"],
    'COMUNA 5':["Almagro" , "Boedo"],
    'COMUNA 6':["Caballito"],
    'COMUNA 7':["Flores" , "Parque Chacabuco"],
    'COMUNA 8':["Villa Soldati", "Villa Riachuelo", "Villa Lugano"],
    'COMUNA 9':["Liniers", "Mataderos" , "Parque Avellaneda"],
    'COMUNA 10':["Villa Real", "Monte Castro", "Versalles", "Floresta", "Vélez Sarfield" , "Villa Luro"],
    'COMUNA 11':["Villa General Mitre", "Villa Devoto", "Villa del Parque" , "Villa Santa Rita"],
    'COMUNA 12':[ "Coghlan", "Saavedra", "Villa Urquiza" ,"Villa Pueyrredón"],
    'COMUNA 13':[ "Núñez", "Belgrano" , "Colegiales"],
    'COMUNA 14':[ "Palermo"],
    'COMUNA 15':[ "Chacarita", "Villa Crespo", "La Paternal", "Villa Ortúzar", "Agronomía" , "Parque Chas"]
}
def asignar_comuna(barrio):
    for comuna, barrios in comunas.items():
        if barrio in barrios:
            return comuna
    return 'Desconocido'  

capi['COMUNA'] = capi['ITE_ADD_NEIGHBORHOOD_NAME'].apply(asignar_comuna)
comunas_grupo = capi.groupby('COMUNA')
for comuna, grupo in comunas_grupo:
    if len(grupo) >= 1500:
        output_dir = f'../Dataset/Barrios/Capital Federal/{comuna}'
        os.makedirs(output_dir)
        grupo.to_csv(f'../Dataset/Barrios/Capital Federal/{comuna}/{comuna}_dev.csv')

os.remove('../Dataset/Barrios/Capital Federal/Capital Federal_dev.csv')

/tmp/ipykernel_9772/5234506.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capi['COMUNA'] = capi['ITE_ADD_NEIGHBORHOOD_NAME'].apply(asignar_comuna)


In [ ]:
#Preprocesamiento
def create_splits_for_barrios(folder_name,comuna=None):
    if comuna is None:
        route = f'../Dataset/Barrios/{folder_name}'
        df = pd.read_csv(f'{route}/{folder_name}_dev.csv')
    else:
        route = f'../Dataset/Barrios/{folder_name}/{comuna}'
        df = pd.read_csv(f'{route}/{comuna}_dev.csv')
    X = df.drop(columns=['precio_pesos_constantes']) 
    y = df['precio_pesos_constantes']  
    y_bins = pd.qcut(y, q=5, labels=False) 
    if comuna is None:
        stratified_split(route,X,y,y_bins,folder_name)
    else:
        stratified_split(route,X,y,y_bins,comuna)

for folder_name in os.listdir('../Dataset/Barrios'):
    if folder_name == 'Capital Federal':
        items = os.listdir(f'../Dataset/Barrios/{folder_name}')
        items = [item.removesuffix("_dev.csv") for item in items]
        for item in items:
          create_splits_for_barrios('Capital Federal',comuna=item)
    else:
        create_splits_for_barrios(folder_name)

#validation

/tmp/ipykernel_9772/414956414.py:8: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{route}/{comuna}_dev.csv')


In [88]:
def create_splits_for_barrios_val(folder_name,comuna=None):
    if comuna is None:
        route = f'../Dataset/Barrios/{folder_name}'
        df = pd.read_csv(f'{route}/{folder_name}_train.csv')
    else:
        route = f'../Dataset/Barrios/{folder_name}/{comuna}'
        df = pd.read_csv(f'{route}/{comuna}_train.csv')
    X = df.drop(columns=['precio_pesos_constantes']) 
    y = df['precio_pesos_constantes']  
    y_bins = pd.qcut(y, q=5, labels=False) 
    if comuna is None:
        stratified_split(route,X,y,y_bins,folder_name,test=False)
    else:
        stratified_split(route,X,y,y_bins,comuna,test=False)

for folder_name in os.listdir('../Dataset/Barrios'):
    if folder_name == 'Capital Federal':
        items = os.listdir(f'../Dataset/Barrios/{folder_name}')
        items = [item.removesuffix("_train.csv") for item in items]
        for item in items:
          create_splits_for_barrios('Capital Federal',comuna=item)
    else:
        create_splits_for_barrios(folder_name)

/tmp/ipykernel_9772/414956414.py:8: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{route}/{comuna}_dev.csv')


In [ ]:
#Preprocesamiento
for folder_name in os.listdir('../Dataset/Barrios'):
    print(folder_name)
    if folder_name == 'Capital Federal':
        for comuna in os.listdir('../Dataset/Barrios/Capital Federal'):
            print(comuna)
            route = f'../Dataset/Barrios/{folder_name}/{comuna}/{comuna}_train.csv'
            df = pd.read_csv(route)
            train_procesado = preprocesar(df, 'train')
            train_procesado.to_csv(f'../Dataset/Barrios/{folder_name}/{folder_name}_train_procesado.csv')
    else:
        route = f'../Dataset/Barrios/{folder_name}/{folder_name}_train.csv'
        df = pd.read_csv(route)
        train_procesado = preprocesar(df, 'train')
        train_procesado.to_csv(f'../Dataset/Barrios/{folder_name}/{folder_name}_train_procesado.csv')

Columna a predecir: Laundry
Columna a predecir: Calefaccion
Columna a predecir: Jacuzzi
Columna a predecir: Gimnasio
Columna a predecir: Cisterna
Columna a predecir: AireAC
Columna a predecir: SalonFiestas
Columna a predecir: Amoblado
Columna a predecir: Antiguedad
Columna a predecir: Laundry
Columna a predecir: Calefaccion
Columna a predecir: Jacuzzi
Columna a predecir: Gimnasio
Columna a predecir: Cisterna
Columna a predecir: AireAC
Columna a predecir: SalonFiestas
Columna a predecir: Amoblado
Columna a predecir: Antiguedad
Columna a predecir: Laundry
Columna a predecir: Calefaccion
Columna a predecir: Jacuzzi
Columna a predecir: Gimnasio
Columna a predecir: Cisterna
Columna a predecir: AireAC
Columna a predecir: SalonFiestas
Columna a predecir: Amoblado
Columna a predecir: Antiguedad
Columna a predecir: Laundry
Columna a predecir: Calefaccion
Columna a predecir: Jacuzzi
Columna a predecir: Gimnasio
Columna a predecir: Cisterna
Columna a predecir: AireAC
Columna a predecir: SalonFies

Traceback (most recent call last):
  File "/home/ciror/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_9772/1236554313.py", line 12, in <module>
    train_procesado = preprocesar(df, 'train')
  File "/home/ciror/Desktop/proyecto_final/proyecto_final_Guerrero_Russi/src/func_preprocesamiento.py", line 26, in preprocesar
    df = preprocesar_binarios(df, binarias, 'RF')
  File "/home/ciror/Desktop/proyecto_final/proyecto_final_Guerrero_Russi/src/func_preprocesamiento.py", line 184, in preprocesar_binarios
    df = valor_faltante_random_forest(df, columna, 'CLAS', False, ind_cols)
  File "/home/ciror/Desktop/proyecto_final/proyecto_final_Guerrero_Russi/src/func_preprocesamiento.py", line 228, in valor_faltante_random_forest
    y_faltantes_pred = modelo.predict(X_faltantes)
  File "/home/ciror/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py", line 823, 